In [ ]:
import numpy as np
import csv
import tensorflow as tf
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
import pandas as pd
from PIL import Image, ImageOps
import h5py
%matplotlib inline

import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

In [ ]:
def identity_block(X, f, filters, stage, block):
    """
    Implementation of the identity block
    
    Arguments:
    X: Input
    f: integer, specifying the shape of the middle CONV's window for the main path
    filters: Number of filters in the CONV layers of the main path
    stage: Integer, used to name the layers, depending on their position in the network
    block: String, used to name the layers, depending on their position in the network
    
    Returns:
    X: Output of the identity block
    """
    # Defining some name basis
    conv_name_base = "res" + str(stage) + block + "_branch"
    bn_name_base = "bn" + str(stage) + block + "_branch"
    
    # Retrieve filters
    F1, F2, F3, F4 = filters
    
    # Save the input value, to use it later to add to the main path
    X_shortcut = X
    
    # first component of main path
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1, 1), padding = "valid", name = conv_name_base + "2a",
               kernel_initializer = glorot_uniform(seed = 0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + "2a")(X)
    X = Activation("relu")(X)
    
    # Second component of main path
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1, 1), padding = "same", name = conv_name_base + "2b",
              kernel_initializer = glorot_uniform(seed = 0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + "2b")(X)
    X = Activation("relu")(X)
    
    # Third component of main path
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1, 1), padding = "valid", name = conv_name_base + "2c",
              kernel_initializer = glorot_uniform(seed = 0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + "2c")(X)
    X = Activation("relu")(X)
    
    # Fourth component of the main path
    X = Conv2D(filters = F4, kernel_size = (f, f), strides = (1, 1), padding = "same", name = conv_name_base + "2d",
              kernel_initializer = glorot_uniform(seed = 0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + "2d")(X)
    X = Activation("relu")(X)
    
    # Add shortcut to the main path and pass it through a RelU activation
    X = Add()([X, X_shortcut])
    X = Activation("relu")(X)
    
    return X

In [ ]:
def convolutional_block(X, f, filters, stage, block, s = 2):
    """
    Implementation of the convolutional block
    
    Arguments:
    X: Input
    f: Integer, specifying the shape of the middle CONV's window for the main path
    filters: filters: Number of filters in the CONV layers of the main path
    stage: Integer, used to name the layers, depending on their position in the network
    block: String, used to name the layers, depending on their position in the network
    s: Integer, specifying the stride to be used
    
    Returns:
    X: Output of the convolutional block
    """
    
    # Defining name basis:
    conv_name_base =  "res" + str(stage) + block + "_branch"
    bn_name_base = "bn" + str(stage) + block + "_branch"
    
    # Retrieve filters
    F1, F2, F3, F4 = filters
    
    # Save the input value
    X_shortcut = X
    
    # First component of main path
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (s, s), padding = "valid", name = conv_name_base + "2a",
              kernel_initializer = glorot_uniform(seed = 0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + "2a")(X)
    X = Activation("relu")(X)
    
    # Second component of main path
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1, 1), padding = "same", name = conv_name_base + "2b",
              kernel_initializer = glorot_uniform(seed = 0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + "2b")(X)
    X = Activation("relu")(X)
    
    #Third component of main path
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (s, s), padding = "valid", name = conv_name_base + "2c",
              kernel_initializer = glorot_uniform(seed = 0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + "2c")(X)
    X = Activation("relu")(X)
    
    #Fourth component of main path
    X = Conv2D(filters = F4, kernel_size = (f, f), strides = (1, 1), padding = "same", name = conv_name_base + "2d",
               kernel_initializer = glorot_uniform(seed = 0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + "2d")(X)
    X = Activation("relu")(X)
    
    # Shortcut path
    X_shortcut = Conv2D(filters = F4, kernel_size = (f, f), strides = (1, 1), padding = "same", name = conv_name_base + "1",
                       kernel_initializer = glorot_uniform(seed = 0))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3, name =  bn_name_base + "1")(X_shortcut)
    
    # Add shortcut to the main path and pass it through a RelU activation
    X = Add()([X, X_shortcut])
    X = Activation("relu")(X)
    
    return X

In [ ]:
def ResNet(input_shape = (512, 512, 3), classes = 1):
    """
    Implementation of a Residual Network with 150 layers
    
    Inputs:
    input_shape: shape of the images
    classes: Integer, number of classes
    
    Returns:
    model: a model() instance in Keras
    """
    
    # Define the input as a tensor
    X_input = Input(input_shape)
    
    # Zero padding
    X = ZeroPadding2D((3, 3))(X_input) 
    
    # Stage 1
    X = Conv2D(64, (7, 7), strides = (2, 2), name = "conv1", kernel_initializer=  glorot_uniform(seed = 0))(X)
    X = BatchNormalization(axis = 3, name = "bn_conv1")(X)
    X = Activation("relu")(X)
    X = MaxPooling2D((3, 3), strides = (2, 2))(X)
    
    # Stage 2
    X = convolutional_block(X, f = 3, filters = [64, 64, 256, 256], stage = 2, block = "a", s = 1)
    X = identity_block(X, 3, [64, 64, 256, 256], stage = 2, block = "b")
    X = identity_block(X, 3, [64, 64, 256, 256], stage = 2, block = "c")
    
    # Stage 3
    X = convolutional_block(X, f = 3, filters = [128, 128, 512, 512], stage = 3, block = "a", s = 1)
    X = identity_block(X, 3, [128, 128, 512, 512], stage = 3, block = "b")
    X = identity_block(X, 3, [128, 128, 512, 512], stage = 3, block = "c")
    X = identity_block(X, 3, [128, 128, 512, 512], stage = 3, block = "d")
    
    # Stage 4
    X = convolutional_block(X, f = 3, filters = [256, 256, 1024, 1024], stage = 4, block = "a", s = 1)
    X = identity_block(X, 3, [256, 256, 1024, 1024], stage = 4, block = "b")
    X = identity_block(X, 3, [256, 256, 1024, 1024], stage = 4, block = "c")
    X = identity_block(X, 3, [256, 256, 1024, 1024], stage = 4, block = "d")
    X = identity_block(X, 3, [256, 256, 1024, 1024], stage = 4, block = "e")
    
    # Stage 5
    X = convolutional_block(X, f = 3, filters = [512, 512, 2048, 2048], stage = 5, block = "a", s = 1)
    X = identity_block(X, 3, [512, 512, 2048, 2048], stage = 5, block = "b")
    X = identity_block(X, 3, [512, 512, 2048, 2048], stage = 5, block = "c")
    X = identity_block(X, 3, [512, 512, 2048, 2048], stage = 5, block = "d")
    X = identity_block(X, 3, [512, 512, 2048, 2048], stage = 5, block = "e")
    X = identity_block(X, 3, [512, 512, 2048, 2048], stage = 5, block = "f")
    
    # Stage 6
    X = convolutional_block(X, f = 3, filters = [1024, 1024, 4096, 4096], stage = 6, block = "a", s = 1)
    X = identity_block(X, 3, [1024, 1024, 4096, 4096], stage = 6, block = "b")
    X = identity_block(X, 3, [1024, 1024, 4096, 4096], stage = 6, block = "c")
    X = identity_block(X, 3, [1024, 1024, 4096, 4096], stage = 6, block = "d")
    
    # Average Pool
    X = AveragePooling2D(pool_size = (2, 2), padding = "same")(X)
    
    # Output layer
    X = Flatten()(X)
    X = Dense(classes, activation = "sigmoid", name = "fc" + str(classes), kernel_initializer = glorot_uniform(seed = 0))(X)
    
    # Create model
    model = Model(inputs = X_input, outputs = X, name = "ResNet")
    
    return model    

In [ ]:
model = ResNet(input_shape = (512, 512, 3), classes = 1)

In [ ]:
model.compile(optimizer='adam', loss='logcosh', metrics=['accuracy'])

In [ ]:
train_data_x = pd.read_csv("MURA-v1.1/train_image_paths.csv")
train_data_y = pd.read_csv("MURA-v1.1/train_labeled_studies.csv")
valid_data_x = pd.read_csv("MURA-v1.1/valid_image_paths.csv")
valid_data_y = pd.read_csv("MURA-v1.1/valid_labeled_studies.csv")
train_x_orig = train_data_x.iloc[:, 0]
train_y_orig = train_data_y.iloc[:, 1]
valid_x_orig = valid_data_x.iloc[:, 0]
valid_y_orig = valid_data_y.iloc[:, 1]

In [ ]:
train_data_array = []
desired_size = 512

for i in range (len(train_x_orig)):
    # Convet images from grayscale to RGB
    image = Image.open(train_x_orig.iloc[i])
    image = image.convert("RGB")
    
    # Resize image so every image got the same shape
    old_size = image.size  # old_size[0] is in (width, height) format
    ratio = float(desired_size)/max(old_size)
    new_size = tuple([int(x*ratio) for x in old_size])
    
    # Use thumbnail() or resize() method to resize the input image  
    image = image.resize(new_size, Image.ANTIALIAS)
    
    # Create a new image and paste the resized on it
    new_image = Image.new("RGB", (desired_size, desired_size))
    new_image.paste(image, ((desired_size-new_size[0])//2,
                    (desired_size-new_size[1])//2))

    array = np.array(new_image)
    
    #Test the dimensions of the data
    print("Shape = " + str(array.shape) + " " + str(i))
    
    train_data_array.append(array)

In [ ]:
# We do it twice, once for training data and another one for validation data
valid_data_array = []
desired_size = 512

for i in range (len(valid_x_orig)):
    image = Image.open(valid_x_orig.iloc[i])
    image = image.convert("RGB")
    
    old_size = image.size  
    ratio = float(desired_size)/max(old_size)
    new_size = tuple([int(x*ratio) for x in old_size])
   
    image = image.resize(new_size, Image.ANTIALIAS)
    
    new_image = Image.new("RGB", (desired_size, desired_size))
    new_image.paste(image, ((desired_size-new_size[0])//2,
                    (desired_size-new_size[1])//2))

    array = np.array(new_image)
    
    print("Shape = " + str(array.shape) + " " + str(i))
    
    valid_data_array.append(array)

In [ ]:
# Stack all the images
train_data = np.stack(train_data_array, axis = 0)
valid_data = np.stack(valid_data_array, axis = 0)

In [ ]:
# Normalize image vectors
X_train = train_data/255
X_valid = valid_data/255
Y_train = train_y_orig
Y_valid = valid_y_orig
print("number of training examples = " + str(X_train.shape[0]))
print("number of test examples = " + str(X_valid.shape[0]))
print("X_train shape: " + str(X_train.shape))
print("Y_train shape: " + str(Y_train.shape))
print("X_test shape: " + str(X_valid.shape))
print("Y_test shape: " + str(Y_valid.shape))

In [ ]:
# Train the model

model.fit(X_train, Y_train, epochs = 3, batch_size = 64)

In [ ]:
preds = model.evaluate(X_valid, Y_valid)
print("Loss = " + str(preds[0]))
print("Test Accuracy = " + str(preds[1]))